In [132]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

In [396]:
samples = pd.read_table("task2_lemmas_train", sep=',',
                        error_bad_lines=False, warn_bad_lines=False)
test = pd.read_table("task2_lemmas_test", sep=',')

In [397]:
print(samples.head())
print(test.head())

   Id             X              y
0   1  vergognerete   vergognare+V
1   2  amnistiavate   amnistiare+V
2   3   menomazione  menomazione+N
3   4    sfaldavamo     sfaldare+V
4   5    sfodererei    sfoderare+V
   Id                X
0   1         gettonan
1   2      incidentali
2   3        involtino
3   4            lievi
4   5  comunistizzasse


In [264]:
# sample = ["abd", "z", "abd", "z", "zm", "www"]
# transformer = CountVectorizer(analyzer='word', token_pattern="(?u)\\b\\w+\\b")
# x = transformer.fit_transform(sample)
# print(x)
# print(transformer.get_feature_names())

In [398]:
transformer = CountVectorizer(analyzer='char', ngram_range=(2, 6))

In [399]:
word_classes = CountVectorizer(analyzer='word', token_pattern="(?u)\\b\\w+\\b")

In [400]:
xtrain, xtest = train_test_split(samples)

In [401]:
X = xtrain['X']
y = xtrain['y']
y = [elem.split("+") for elem in y]
y_norm, y_form = zip(*y)

In [402]:
train_matrix = transformer.fit_transform(X)

In [270]:
# def cl(x):
#     if x == 'a':
#         return 0
#     if x == 'b':

In [271]:
# train_target = [cl(yy) for yy in y_form]

In [403]:
predictor = LogisticRegression().fit(train_matrix, y_form)

In [273]:
X = xtest['X']
y = xtest['y']
y = [elem.split("+") for elem in y]
y_norm, y_form = zip(*y)

In [274]:
test_matrix = transformer.transform(X)

In [275]:
preds = predictor.predict_proba(test_matrix)

In [276]:
roc_auc_score(word_classes.fit_transform(y_form).toarray(), preds)

0.99254495293519829

# Endings

In [277]:
def endings_classifier(X, y_norm):
    endings = []
    for i in range(X.shape[0]):
#         print(type(X[i]))
        x = X[i]
#         print(X[i])
        y = y_norm[i]
        stem = 0
        for j in range(len(x)):
            if j < len(y):
                if y[j] == x[j]:
                    stem = stem + 1
        x_end = x[stem:]
        y_end = y[stem:]
        line = str(str(stem) + " " + y_end)
        endings.append(line)
    return endings

In [316]:
X = xtrain['X']
y = xtrain['y']
y = [elem.split("+") for elem in y]
y_norm, y_form = zip(*y)

In [317]:
train_matrix = transformer.fit_transform(X)

In [326]:
endings_transformation = CountVectorizer(analyzer='word', token_pattern=".+")

In [327]:
X = np.array(X)
y_new = endings_classifier(X, y_norm)
endings_transformation.fit_transform(y_new)
print(len(endings_transformation.get_feature_names()))
print(endings_transformation.get_feature_names()[:10])

296
['0 essere', '0 grande', '1 enire', '1 ettere', '1 io', '1 orire', '1 ssere', '1 uocere', '10 ', '10 a']


In [282]:
predictor = LogisticRegression().fit(train_matrix, y_new)

In [344]:
X = xtest['X']
y = xtest['y']
y = [elem.split("+") for elem in y]
y_norm, y_form = zip(*y)

In [345]:
test_matrix = transformer.transform(X)

In [349]:
preds = predictor.predict_proba(test_matrix)

In [350]:
y_new_test = endings_classifier(np.array(xtest['X']), y_norm)

In [351]:
print(type(endings_transformation.transform(y_new_test).toarray()))
print(type(preds))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [355]:
y_true = endings_transformation.transform(y_new_test).toarray()

(29120, 296) (29120, 296)


In [359]:
X = test['X']

In [360]:
test_matrix = transformer.transform(X)

In [361]:
preds = predictor.predict_proba(test_matrix)

In [362]:
names = endings_transformation.get_feature_names()

In [373]:
a = [list(zip(preds[i], names)) for i in range(len(preds))]

In [383]:
b = [max(x)[1] for x in a]

In [388]:
c = [int(x.split(" ")[0]) for x in b]

In [389]:
d = [x.split(" ")[1] for x in b]

In [392]:
ans = [X[i][:c[i]] + d[i] for i in range(len(X))]

In [405]:
X = test['X']

In [406]:
test_matrix = transformer.transform(X)

In [407]:
preds = predictor.predict_proba(test_matrix)

In [408]:
a = [list(zip(preds[i], ['a', 'n', 'v'])) for i in range(len(preds))]

In [412]:
b = [max(x)[1].upper() for x in a]

In [413]:
print(b[:10])

['V', 'A', 'V', 'N', 'V', 'V', 'V', 'V', 'V', 'V']


In [414]:
final = [ans[i] + "+" + b[i] for i in range(len(b))]

In [416]:
preds = final

In [419]:
submission = pd.read_csv("task2_lemmas_sample_submission (1)")
print(len(preds), submission.shape[0])
submission['Category'] = preds
submission.head(5)
submission.to_csv("baseline_submission.tsv", sep=',', index=False)

29661 29661
